In [1]:
from collections import defaultdict

puzzle = defaultdict(int)

with open("inputs/day19-input.txt") as f:
    for key, value in enumerate(map(int, f.read().split(","))):
        puzzle[key] = value

## Part 1

In [2]:
%%time

from typing import Dict, List

class IntcodeComputer:
    def __init__(self, intcode: Dict[int, int]):
        self.intcode = intcode
        self.input = []

        
    def get_param_index(self, instruction_index: int, parameter: int) -> int:
        instruction = f"{self.intcode[instruction_index]:05d}"
        mode = instruction[-2 - parameter]
        is_position  = (mode == "0")
        is_immediate = (mode == "1")
        is_relative  = (mode == "2")
        
        param_index = instruction_index + parameter
        param_value = self.intcode[param_index]
        
        if is_position:
            return param_value
        
        elif is_immediate:
            return param_index
        
        elif is_relative:
            return self.relative_base + param_value
        
    def get_param(self, instruction_index: int, parameter: int) -> int:
        return self.intcode[self.get_param_index(instruction_index, parameter)]


    def get_opcode(self, instruction_index: int) -> int:
        opcode = str(self.intcode[instruction_index])[-2:]
        return int(opcode)


    def run_program(self) -> int:
        index = 0
        self.relative_base = 0
        
        while True:
            opcode = self.get_opcode(index)

            if opcode == 99:
                yield True
                break

            if opcode == 1:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = in1 + in2
                index += 4

            elif opcode == 2:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = in1 * in2
                index += 4

            elif opcode == 3:
                in1 = self.get_param_index(index, 1)

                self.intcode[in1] = self.input.pop(0)
                index += 2

            elif opcode == 4:
                in1 = self.get_param(index, 1)

                self.result = in1
                yield False
                index += 2

            elif opcode == 5:
                in1 = self.get_param(index, 1)
                if in1:
                    in2 = self.get_param(index, 2)
                    index = in2
                else:
                    index += 3

            elif opcode == 6:
                in1 = self.get_param(index, 1)
                if not in1:
                    in2 = self.get_param(index, 2)
                    index = in2
                else:
                    index += 3

            elif opcode == 7:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = int(in1 < in2)
                index += 4

            elif opcode == 8:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = int(in1 == in2)
                index += 4
                
            elif opcode == 9:
                in1 = self.get_param(index, 1)
                
                self.relative_base += in1
                index += 2
    

results = []
for y in range(50):
    for x in range(50):
        computer = IntcodeComputer(puzzle.copy())
        program = computer.run_program()
        computer.input.append(x)
        computer.input.append(y)
        next(program)
        results.append(computer.result)
        
print(results.count(1))

for i, t in enumerate(results):
    if i and i % 50 == 0:
        print()
    print(t, end="")
print()

158
10000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000
01000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000
00100000000000000000000000000000000000000000000000
00010000000000000000000000000000000000000000000000
00010000000000000000000000000000000000000000000000
00001000000000000000000000000000000000000000000000
00001000000000000000000000000000000000000000000000
00000100000000000000000000000000000000000000000000
00000110000000000000000000000000000000000000000000
00000010000000000000000000000000000000000000000000
00000011000000000000000000000000000000000000000000
00000001100000000000000000000000000000000000000000
00000001100000000000000000000000000000000000000000
00000000110000000000000000000000000000000000000000
00000000110000000000000000000000000000000000000000
00000000011000000000000000000000000000000000000000
00000000011100000000000000000000000000000000000000
000000000011000000000000000

In [3]:
%%time

def drone(x, y):
    computer = IntcodeComputer(puzzle.copy())
    program = computer.run_program()
    computer.input.append(x)
    computer.input.append(y)
    next(program)
    return computer.result


min_x = 0
max_x = 0
y = 50

while True:
    while not drone(min_x, y):
        min_x += 1
    
    max_x = min_x + 99
    if drone(max_x, y):
        if drone(min_x, y - 99) and drone(max_x, y - 99):
            print(min_x * 10_000 + (y - 99))
            break
            
    y += 1

6191165
CPU times: user 3.63 s, sys: 3.59 ms, total: 3.64 s
Wall time: 3.65 s
